# Fight with all the imports for CMIP6 data

In [ ]:
# google colab installs

!pip install condacolab &> /dev/null
import condacolab
condacolab.install()

# install all packages in one call (+ use mamba instead of conda)
!mamba install xarray-datatree intake-esm gcsfs xmip aiohttp cartopy nc-time-axis cf_xarray xarrayutils "esmf<=8.3.1" xesmf &> /dev/null

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [ ]:
# installations ( uncomment and run this cell ONLY when using google colab or kaggle )

# # properly install cartopy in colab to avoid session crash
!apt-get install libproj-dev proj-data proj-bin --quiet
!apt-get install libgeos-dev --quiet
!pip install cython --quiet
!pip install cartopy --quiet
!pip install geoviews

!apt-get -qq install python-cartopy python3-cartopy  --quiet
!pip uninstall -y shapely  --quiet
!pip install shapely --no-binary shapely  --quiet

!pip install boto3 --quiet

# you may need to restart the runtime after running this cell and that is ok

Reading package lists...
Building dependency tree...
Reading state information...
libproj-dev is already the newest version (8.2.1-1).
libproj-dev set to manually installed.
proj-data is already the newest version (8.2.1-1).
proj-data set to manually installed.
The following NEW packages will be installed:
  proj-bin
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 154 kB of archives.
After this operation, 492 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 proj-bin amd64 8.2.1-1 [154 kB]
Fetched 154 kB in 1s (114 kB/s)
Selecting previously unselected package proj-bin.
(Reading database ... 120901 files and directories currently installed.)
Preparing to unpack .../proj-bin_8.2.1-1_amd64.deb ...
Unpacking proj-bin (8.2.1-1) ...
Setting up proj-bin (8.2.1-1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists...
Building dependency tree...
Reading state information...
libgeos-dev is alrea

In [ ]:
# imports

import time

tic = time.time()

import intake
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
from xmip.preprocessing import combined_preprocessing
from xarrayutils.plotting import shaded_line_plot
from datatree import DataTree
from xmip.postprocessing import _parse_metric
import cartopy.crs as ccrs
import pooch
import os
import tempfile

In [ ]:
# print(intake.registry)

<Intake driver registry>


In [ ]:
# import sys
# print(sys.executable)

/usr/bin/python3.real


In [ ]:
# functions

%matplotlib inline

col = intake.open_esm_datastore(
    "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
)  # open an intake catalog containing the Pangeo CMIP cloud data


def load_cmip6(source_id, variable_id, member_id, table_id):  # load selected model
    cat = col.search(
        source_id=source_ids,
        variable_id=variable_id,
        member_id=member_id,
        table_id=table_id,
        grid_label="gn",
        experiment_id=[
            "historical",
            "ssp126",
            "ssp245",
            "ssp585",
        ],  # downloading the scenarios out of the total 5+historical
        require_all_on=["source_id"],
    )

    kwargs = dict(
        preprocess=combined_preprocessing,
        xarray_open_kwargs=dict(use_cftime=True),
        storage_options={"token": "anon"},
    )
    cat.esmcat.aggregation_control.groupby_attrs = ["source_id", "experiment_id"]
    dt = cat.to_datatree(**kwargs)

    return dt

/usr/local/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)


In [ ]:
# helper functions

def pooch_load(filelocation=None,filename=None,processor=None):
    shared_location='/home/jovyan/shared/Data/Projects/ENSO' # this is different for each day
    user_temp_cache=tempfile.gettempdir()

    if os.path.exists(os.path.join(shared_location,filename)):
        file = os.path.join(shared_location,filename)
    else:
        file = pooch.retrieve(filelocation,known_hash=None,fname=os.path.join(user_temp_cache,filename),processor=processor)

    return file

# NOAA Dataset

In [ ]:
# Ocean surface temprature
filename_SST='sst.mnmean.nc'
url_SST = 'https://downloads.psl.noaa.gov/Datasets/noaa.ersst.v5/sst.mnmean.nc'

do_sst = xr.open_dataset(pooch_load(url_SST,filename_SST), drop_variables=['time_bnds'])

# Precipitation rate (notice the units in the plot below)
filename_prec_rate='precip.mon.mean.nc'
url_prec_rate='https://downloads.psl.noaa.gov/Datasets/cmap/enh/precip.mon.mean.nc'
do_pr = xr.open_dataset(pooch_load(url_prec_rate,filename_prec_rate))

# Air Temperature Anomalies
filename_tas='air.2x2.1200.mon.anom.comb.nc'
url_tas='https://downloads.psl.noaa.gov/Datasets/gistemp/combined/1200km/air.2x2.1200.mon.anom.comb.nc'
do_tas = xr.open_dataset(pooch_load(url_tas,filename_tas))

SHA256 hash of downloaded file: 94b6b7236e2cb54118ac2ba1dc10ba3f06850fd3a680e089d48ed22d353d0500
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 0224151adc706cfcf88b6be5739a4d380b821e448da02ff732bb59182941cc5b
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 7c7bcc5200abb60fd9ed57c1a446590fc988eecd704aa3a9c224d33b39a37f62
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


# CMIP6 Dataset

In [ ]:
cat = col.search(
    source_id="GFDL-ESM4",
    variable_id=[
        # "gpp",
        # "npp",
        # "nbp",
        # "treeFrac",
        # "grassFrac",
        "cropFrac",
        # "pastureFrac",
    ],  # No 'shrubFrac','baresoilFrac','residualFrac' in GFDL-ESM4
    member_id="r1i1p1f1",
    table_id="Lmon", #["Oclim", "Amon", "Omon", "Lmon", "Limon", "Oimon", "Aero", "cfMon"]
    grid_label="gr1",
    experiment_id=["historical"],
    require_all_on=[
        "source_id"
    ],  # make sure that we only get models which have all of the above experiments
)

/usr/local/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/usr/local/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/usr/local/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)


In [ ]:
# convert the sub-catalog into a datatree object, by opening each dataset into an xarray.Dataset (without loading the data)
kwargs = dict(
    preprocess=combined_preprocessing,  # apply xMIP fixes to each dataset
    xarray_open_kwargs=dict(
        use_cftime=True
    ),  # ensure all datasets use the same time index
    storage_options={
        "token": "anon"
    },  # anonymous/public authentication to google cloud storage
)

cat.esmcat.aggregation_control.groupby_attrs = ["source_id", "experiment_id"]
dt_Lmon_variables = cat.to_datatree(**kwargs)

# convert to dataset instead of datatree, remove extra singleton dimensions
ds_Lmon = dt_Lmon_variables["GFDL-ESM4"]["historical"].to_dataset().squeeze()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'source_id/experiment_id'


In [ ]:
ds = xr.Dataset()

# add land variables
for var in ds_Lmon.data_vars:
    ds[var] = ds_Lmon[var]


In [ ]:
# drop unnecessary coordinates
ds = ds.drop_vars(["member_id", "dcpp_init_year"])
ds

<xarray.Dataset>
Dimensions:   (y: 180, x: 288, time: 1980)
Coordinates:
  * y         (y) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * x         (x) float64 0.625 1.875 3.125 4.375 ... 355.6 356.9 358.1 359.4
  * time      (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    lon       (x, y) float64 0.625 0.625 0.625 0.625 ... 359.4 359.4 359.4 359.4
    lat       (x, y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Data variables:
    cropFrac  (time, y, x) float32 dask.array<chunksize=(901, 180, 288), meta=np.ndarray>

# Get the monthly mean for Indonesia only

In [ ]:
# take the timeseries
## Coordinates Indonesia

x_ind = slice(95, 141) # longitude
y_ind = slice(7,-12) # latitude

## SST

sst_timeseries = do_sst.sel(time=slice('1979-07-01','2014-12-01'), lat=y_ind, lon=x_ind)
sst_timeseries = sst_timeseries.mean(["lon", "lat"], keep_attrs=True)
sst_timeseries_mean = np.array(sst_timeseries.sst)

## Air temp anomalies

tas_timeseries = do_tas.sel(time=slice('1979-07-01','2014-12-01'), lat=y_ind, lon=x_ind)
tas_timeseries = tas_timeseries.mean(["lon", "lat"], keep_attrs=True)
tas_timeseries_mean = np.array(tas_timeseries.air)

## Precipitation

rain_timeseries = do_pr.sel(time=slice('1979-07-01','2014-12-01'), lat=y_ind, lon=x_ind)
rain_timeseries = rain_timeseries.mean(["lon", "lat"], keep_attrs=True)
rain_timeseries_mean = np.array(rain_timeseries.precip)

## Crop fraction

x_ind = slice(95, 141) # longitude
y_ind = slice(-12,7) # latitude

crop_frac_timeseries = ds.sel(time=slice('1979-07-01','2014-12-16'), y=y_ind, x=x_ind)
crop_frac_timeseries = crop_frac_timeseries.mean(['y','x'], keep_attrs=True)
crop_frac_timeseries_mean = np.array(crop_frac_timeseries.cropFrac)

In [ ]:
## Convert the xarray.datasets into pd.DataFrame
from google.colab import files

df_tas = tas_timeseries.to_dataframe()
df_tas.to_csv('df_tas.csv', encoding = 'utf-8-sig')
files.download('df_tas.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_sst = sst_timeseries.to_dataframe()
df_sst.to_csv('df_sst.csv', encoding = 'utf-8-sig')
files.download('df_sst.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_rain = rain_timeseries.to_dataframe()
df_rain.to_csv('df_rain.csv', encoding = 'utf-8-sig')
files.download('df_rain.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

df_crop = crop_frac_timeseries.to_dataframe()
df_crop.to_csv('df_crop.csv', encoding = 'utf-8-sig')
files.download('df_crop.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_tas

,air
time,
1979-07-01,-0.002713
1979-08-01,0.127101
1979-09-01,0.149556
1979-10-01,0.156329
1979-11-01,0.293725
...,...
2014-08-01,0.530481
2014-09-01,0.503787
2014-10-01,0.550930


In [ ]:
df_sst

,sst
time,
1979-07-01,27.735752
1979-08-01,27.539595
1979-09-01,27.833902
1979-10-01,28.360090
1979-11-01,28.893454
...,...
2014-08-01,28.089165
2014-09-01,28.228844
2014-10-01,28.645199


In [ ]:
df_rain

,precip
time,
1979-07-01,5.431944
1979-08-01,3.386111
1979-09-01,5.865000
1979-10-01,5.342500
1979-11-01,8.013820
...,...
2014-08-01,5.148611
2014-09-01,3.842014
2014-10-01,4.140972


In [ ]:
df_crop

,cropFrac
time,
1979-07-16 12:00:00,2.936973
1979-08-16 12:00:00,2.936973
1979-09-16 00:00:00,2.936973
1979-10-16 12:00:00,2.936973
1979-11-16 00:00:00,2.936973
...,...
2014-08-16 12:00:00,5.291160
2014-09-16 00:00:00,5.291160
2014-10-16 12:00:00,5.291160


# Download the DeepAR package

In [ ]:
pip install "gluonts[torch]"

In [ ]:
pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset

/usr/local/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


# Run DeepAR

## Prepare the data

In [2]:
import pandas as pd
df_crop = pd.read_csv('df_crop.csv')
df_rain = pd.read_csv('df_rain.csv')
df_sst = pd.read_csv('df_sst.csv')
df_tas = pd.read_csv('df_tas.csv')

In [3]:
df_crop['new_time'] = pd.to_datetime(df_rain['time'])
df_crop.index = df_crop['new_time']
df_crop = df_crop.drop(columns='time',axis=1)


In [4]:
df_tas['time'] = pd.to_datetime(df_tas['time'])
df_tas.index = df_tas['time']
df_tas = df_tas.drop(columns='time',axis=1)

df_rain['time'] = pd.to_datetime(df_rain['time'])
df_rain.index = df_rain['time']
df_rain = df_rain.drop(columns='time',axis=1)

df_sst['time'] = pd.to_datetime(df_sst['time'])
df_sst.index = df_sst['time']
df_sst = df_sst.drop(columns='time',axis=1)




In [ ]:
# df_tas['time'] = df_tas['time'].to_timestamp
# df_sst['time'] = df_sst['time'].str[:-3]
# df_rain['time'] = df_rain['time'].str[:-3]
# df_crop['time'] = df_crop['time'].str[:-12]

In [ ]:
# df_crop.index = df_crop.time
# df_crop = df_crop.drop(columns= 'time')
# df_crop.columns

Index(['cropFrac'], dtype='object')

In [ ]:
# df_rain.index = df_rain.time
# df_rain = df_rain.drop(columns= 'time')
# df_rain.columns

Index(['precip'], dtype='object')

In [ ]:
# df_sst.index = df_sst.time
# df_sst = df_sst.drop(columns= 'time')
# df_sst.columns

Index(['sst'], dtype='object')

In [ ]:
# df_tas.index = df_tas.time
# df_tas = df_tas.drop(columns= 'time')
# df_tas.columns

Index(['air'], dtype='object')

In [ ]:
# type(df_crop.index[0])

str

In [5]:
#from gluonts.dataset.pandas import PandasDataset
df_final = pd.concat([df_tas, df_rain, df_sst['sst'], df_crop['cropFrac']], axis=1)
df_final.head()

,air,precip,sst,cropFrac
1979-07-01,-0.002713,5.431944,27.735752,2.936973
1979-08-01,0.127101,3.386111,27.539595,2.936973
1979-09-01,0.149556,5.865000,27.833902,2.936973
1979-10-01,0.156329,5.342500,28.360090,2.936973
1979-11-01,0.293725,8.013820,28.893454,2.936973


In [ ]:
from gluonts.dataset.pandas import PandasDataset

ds = PandasDataset(dict(df_tas))

## Run the model

In [ ]:
import mxnet as mx
import torch
from gluonts.mx import DeepAREstimator
from gluonts.mx.trainer import Trainer
import numpy as np

In [ ]:
def set_seed(seed):
    import os
    import random
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    mx.random.seed(seed)

set_seed(42)
estimator = DeepAREstimator(
    prediction_length=12,
    freq="M",
    trainer=Trainer(
        learning_rate=1e-3, epochs=10, num_batches_per_epoch=5
    ),
    # use_feat_static_cat=True,
    # cardinality=[len(cfip2idx), len(state2idx), len(county2idx)],
    use_feat_static_real=True
)

In [ ]:
from gluonts.mx import DeepAREstimator, Trainer


In [ ]:
estimator = DeepAREstimator(
    freq=ds.freq, prediction_length=12, trainer=Trainer(epochs=1)
)

In [ ]:
## should we run it with instead train and test split?

In [ ]:
predictor = estimator.train(ds)
predictions = predictor.predict(ds)

100%|██████████| 50/50 [00:06<00:00,  7.91it/s, epoch=1/1, avg_epoch_loss=0.0228]


#### Evaluation of the predictions

In [ ]:
from gluonts.evaluation import make_evaluation_predictions

forecast_it, ts_it = make_evaluation_predictions(
    dataset=ds,  # test dataset
    predictor=predictor,  # predictor
    num_samples=500,  # number of sample paths we want for evaluation
)

# we convert these generators to lists to ease the subsequent computations.
forecasts = list(forecast_it)
tss = list(ts_it)

In [ ]:
import json
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(tss, forecasts)
print(json.dumps(agg_metrics, indent=4))

Running evaluation: 1it [00:00, 18.82it/s]

{
    "MSE": 0.05097930310385931,
    "abs_error": 2.1523009253961183,
    "abs_target_sum": 8.31607827,
    "abs_target_mean": 0.6930065225,
    "seasonal_error": 0.2787009845873134,
    "MASE": 0.6435514058741542,
    "MAPE": 0.23595615916218038,
    "sMAPE": 0.2669132696058974,
    "MSIS": 4.6413350209692625,
    "QuantileLoss[0.1]": 1.1478674073063888,
    "Coverage[0.1]": 0.0,
    "QuantileLoss[0.5]": 2.1523009253961183,
    "Coverage[0.5]": 0.3333333333333333,
    "QuantileLoss[0.9]": 0.6898252564443359,
    "Coverage[0.9]": 0.8333333333333334,
    "RMSE": 0.22578596746445362,
    "NRMSE": 0.3258064103782711,
    "ND": 0.25881200916067354,
    "wQuantileLoss[0.1]": 0.13802989462560564,
    "wQuantileLoss[0.5]": 0.25881200916067354,
    "wQuantileLoss[0.9]": 0.08295078930808764,
    "mean_absolute_QuantileLoss": 1.3299978630489477,
    "mean_wQuantileLoss": 0.15993089769812227,
    "MAE_Coverage": 0.3777777777777778,
    "OWA": NaN
}


In [ ]:
# from gluonts.dataset.field_names import FieldName
# from gluonts.dataset.common import ListDataset
# from gluonts.dataset.pandas import PandasDataset
# from gluonts.dataset.util import to_pandas


# Run LSTM

In [ ]:
## let's remember that we have just created one unique dataframe with all our variables called with the surprising name
## of df_final because we have such a great fantasy no

df_train = df_final.iloc[:414,:]
df_test = df_final.iloc[414:,:]

In [ ]:
df_train.shape, df_test.shape

((414, 4), (12, 4))

In [ ]:
import sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
df_train_scaled = sc.fit_transform(df_train)
df_train_scaled.shape

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

(414, 4)

# Fake data:
- SDV
- Knockoffs

In [6]:
## the data that we are going to use is the one that we just used before in order to run the DeepAR

df_final.describe()

,air,precip,sst,cropFrac
count,426.000000,426.000000,426.000000,426.000000
mean,0.407676,6.598593,28.702224,4.011410
std,0.263519,2.087461,0.517318,0.644612
min,-0.269946,1.647708,27.213500,2.936973
25%,0.218853,5.192188,28.361538,3.557627
50%,0.396895,6.348785,28.784728,3.931068
75%,0.584939,7.928525,29.058912,4.538285
max,1.218662,12.408543,29.826021,5.291160


## SDV
From https://ealizadeh.com/blog/sdv-library-for-modeling-datasets/

In [ ]:
# pip install sdv

In [ ]:
# pip install timeseries

In [ ]:
# import sdv
# print(sdv.__version__)

1.4.0


In [ ]:
# #import pandas as pd
# import sdv
# from sdv.timeseries import PAR

from sdv.single_table import GaussianCopulaSynthesizer


In [ ]:
df_final.dtypes

air         float64
precip      float64
sst         float64
cropFrac    float64
dtype: object

In [ ]:
df_final.head()

,air,precip,sst,cropFrac
1979-07-01,-0.002713,5.431944,27.735752,2.936973
1979-08-01,0.127101,3.386111,27.539595,2.936973
1979-09-01,0.149556,5.865000,27.833902,2.936973
1979-10-01,0.156329,5.342500,28.360090,2.936973
1979-11-01,0.293725,8.013820,28.893454,2.936973


## Knockoffs

In [19]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [8]:
%%R
x <- 42
print(x)

[1] 42


In [12]:
%%R

set.seed(1234)



Error in library(knockoff) : there is no package called ‘knockoff’


In [ ]:
%%R
## Example on how to generate X-fixed knockoffs

n = 1000          # number of observations
p = 300           # number of variables
k = 30            # number of variables with nonzero coefficients
amplitude = 4.5   # signal amplitude (for noise level = 1)

# Generate the variables from a multivariate normal distribution
mu = rep(0,p)
rho = 0.25
Sigma = toeplitz(rho^(0:(p-1)))
X = matrix(rnorm(n*p),n) %*% chol(Sigma)

# Generate the response from a linear model
nonzero = sample(p, k)
beta = amplitude * (1:p %in% nonzero) / sqrt(n)
y.sample = function(X) X %*% beta + rnorm(n)
y = y.sample(X)
summary(X)

In [ ]:
%%R

install.packages("knockoff")

#library(knockoff)


In [ ]:
# %%R
# library(knockoff)
# library(tidyverse)
# library(psych)

In [25]:
%%R
summary(y)


       V1          
 Min.   :-3.57802  
 1st Qu.:-0.91850  
 Median :-0.01978  
 Mean   :-0.02352  
 3rd Qu.: 0.81827  
 Max.   : 3.88768  


In [ ]:
%%R
result = knockoff.filter(X, y)

In [29]:
%%R
print(result$selected)

 [1]   5   6   7  10  21  22  33  34  35  36  48  96 114 122 127 141 145 181 187
[20] 208 269 298


In [ ]:
%%R

create.fixed(
      X,
      method = c("sdp", "equi"),
      sigma = NULL,
      y = NULL,
      randomize = F
)

## Deep Knockoffs

In [ ]:
## just an example on how to install github repositories as packages
# !pip install git+https://github.com/powerscorinne/GLRM.git

In [ ]:
# !pip install git+https://github.com/msesia/deepknockoffs/tree/98588cc326730bd8ca45504ab7e52a42269effc2/DeepKnockoffs

In [ ]:
 ## alternative way to install a github repo as a package

#!git clone https://github.com/msesia/deepknockoffs.git

fatal: destination path 'deepknockoffs' already exists and is not an empty directory.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from deepknockoffs import DeepKnockoffs as d

In [ ]:
!python /content/deepknockoffs/DeepKnockoffs/setup.py

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based t

In [ ]:
from deepknockoffs import KnockoffMachine

ImportError: ignored